# ERDRI CDS
The "Set of common data elements for Rare Diseases Registration" is the first practical instrument released by the EU RD Platform aiming at increasing interoperability of RD registries.

It contains 16 data elements to be registered by each rare disease registry across Europe, which are considered to be essential for further research. They refer to patient's personal data, diagnosis, disease history and care pathway, information for research purposes and about disability.

The "Set of common data elements for Rare Diseases Registration" was produced by a Working Group coordinated by the JRC and composed of experts from EU projects which worked on common data sets: EUCERD Joint Action, EPIRARE and RD-Connect.

[Source](https://eu-rd-platform.jrc.ec.europa.eu/set-of-common-data-elements_en)

## 1. Defining the ERDRI CDS Data Model
Instead of loading in a data model from a file, we can also create a data model by defining it in code. This skips the parsing step and allows us to define the data model in a more flexible way.

![ERDRI CDS](../res/imgs/notebooks/erdri_cds1.png)
![ERDRI CDS](../res/imgs/notebooks/erdri_cds2.png)

### 1.1. Resources

Every data model in the modern digital health environment will probably rely on at least one or more code systems. Code systems might include terminologies, classifications, or ontologies. Popular code systems include LOINC, SNOMED CT, ICD-10, and Orphanet Rare Disease Ontology (ORDO).

In this case, we will use the following code systems (in order as listed in the CODING column of the ERDRI CDS data model above):
- [ORDO](https://www.orpha.net/en/disease): Orphanet Rare Disease Ontology
- [Alpha-ID-SE](https://www.bfarm.de/EN/Code-systems/Terminologies/Alpha-ID-SE/_node.html): Simplified, uniform and standardised coding of rare diseases according to ICD-10-GM
- [ICD-9](https://iris.who.int/handle/10665/39473): International Classification of Diseases, Ninth Revision
- [ICD-9-CM](https://archive.cdc.gov/www_cdc_gov/nchs/icd/icd9cm.htm#:~:text=ICD%2D9%2DCM%20is%20the,10%20for%20mortality%20coding%20started.): International Classification of Diseases, Ninth Revision, Clinical Modification
- [ICD-10](https://www.who.int/classifications/icd/en/): International Classification of Diseases, Tenth Revision
- [HGVS](https://hgvs.org/): Human Genome Variation Society
- [HGNC](https://www.genenames.org/): HUGO Gene Nomenclature Committee
- [OMIM](https://www.omim.org/): Online Mendelian Inheritance in Man
- [HPO](https://hpo.jax.org/): Human Phenotype Ontology


There are already some popular code systems predefined in the `phenopacket_mapper` package. We can use them directly in our data model. Although optional, it is recommended to add the correct or most recent version to each code system. This will help to ensure that the code system is correctly identified and used in the data model.

The versions added below are the most recent at the time of writing this notebook. You can check for the most recent version of each code system on the respective website.

In [66]:
import phenopacket_mapper

In [67]:
from phenopacket_mapper.data_standards.code_system import ORDO, ICD9, HGVS, HGNC, OMIM, HPO 

In [68]:
resources = [
    ORDO.set_version("1.0.19 (2024-08-02)"),
    ICD9,
    HGVS.set_version("21.0.4 (2024-08-15)"),
    HGNC.set_version("2024-08-23"),
    OMIM.set_version("2024-09-12"),
    HPO.set_version("2024-06-07")
]

The keen eyed among you might have spotted that we forgot to add the ICD-9-CM, ICD-10, and Alpha-ID-SE code systems. We can add them to the list of resources by creating a new `CodeSystem` object for each of them. We can then add them to the `resources` list.:

In [69]:
from phenopacket_mapper.data_standards.code_system import CodeSystem

In [70]:
alpha = CodeSystem(name='Alpha-ID-SE', namespace_prefix='alpha', url='https://www.bfarm.de/EN/Code-systems/Terminologies/Alpha-ID-SE/_node.html')
icd9cm = CodeSystem(name='International Classification of Diseases 9 Clinical Modification (USA)', namespace_prefix='icd-9-cm', url='http://hl7.org/fhir/sid/icd-9-cm')
icd10 = CodeSystem(name='International Classification of Diseases 10 (WHO)', namespace_prefix='icd-10', url='http://hl7.org/fhir/sid/icd-10')

In [71]:
resources.append(alpha)
resources.append(icd9cm)
resources.append(icd10)

### 1.2 Fields of the data model and their value sets

As you can see above, the ERDRI CDS are made up of 8 sections with a total of 16 fields. Each field has a name, a description, and a value set. The value set is a list of possible values that the field can take, such as a list of codings, strings or numerical values to choose from. Another option is to restrict values in a field to a possible data type (e.g., string), code system (e.g., ORDO), a date, etc.

However, to limit the complexity of this example, we will omit the 7th and 8th sections, as they are also difficult to model in phenopackets. We will focus on the first 6 sections, containing 11 fields.

We will start here by defining the value sets of all the fields in the ERDRI CDS data model. We will use the code systems we defined above to define the value sets of the fields.

In [72]:
from phenopacket_mapper.data_standards.value_set import ValueSet

We will start by defining the value set for the first field of the ERDRI CDS data model, which is the pseudonym. The pseudonym is a string, so we will define a value set with the element type set to `str`. We will also add a name and description to the value set to make it easier to identify later on.

In [73]:
# 1. Pseudonym
# 1.1. Pseudonym
vs_1_1 = ValueSet(
    elements=[str],
    name="Value set for 1.1. Pseudonym",
    description="Value set for field 1.1. Pseudonym of the ERDRI CDS data model in section 1. Pseudonym",
)

Next, we get to the section about personal information. Here the first field is the date of birth. The date of birth is a date, so we will define a value set with the element type set to the `Date` type provided by `phenopacket-mapper`.

In [74]:
from phenopacket_mapper.data_standards import Date

In [75]:
# 2. Personal information
# 2.1. Date of Birth
vs_2_1 = ValueSet(
    elements=[Date],
    name="Value set for 2.1. Date of Birth",
    description="Value set for field 2.1. Date of Birth of the ERDRI CDS data model in section 2. Personal information",
)

The next field: "Sex" is a categorical field with four possible values: 
- Female,
- Male,
- Undetermined, and
- Foetus (Unknown). 

It is to be noted that it is generally recommended to encode concepts with an internationally recognized code system. However, in this case the authors of the ERDRI CDS data model have not specified a code system for the field, making things easier for us. 

The same can be said for field 3.1.

At the bottom of this notebook there is an examplary implementation of this field using concepts from the SNOEMD CT code system. The example also introduces you to `CodeableConcept` objects.

In [76]:
# 2.2. Sex
vs_2_2 = ValueSet(
    elements=["Female", "Male", "Undetermined", "Foetus (Unknown)"],
    name="Value set for 2.2. Sex",
    description="Value set for field 2.2. Sex of the ERDRI CDS data model in section 2. Personal information",
)
# 3. Patient Status
# 3.1. Patient's status
vs_3_1 = ValueSet(
    elements=["Alive", "Dead", "Lost in follow-up", "Opted-out"],
    name="Value set for 3.1. Patient's status",
    description="Value set for field 3.1. Patient's status of the ERDRI CDS data model in section 3. Patient Status",
)

The field date of death relies on a date value, so we will define a value set with the element type set to the `Date` type provided by `phenopacket-mapper`.

In [77]:
# 3.2. Date of death
vs_3_2 = ValueSet(
    elements=[Date],
    name="Value set for 3.2. Date of death",
    description="Value set for field 3.2. Date of death of the ERDRI CDS data model in section 3. Patient Status",
)

Note that we could have also just created a single value set for the date fields and reused it for all the date fields. This would have been more efficient and would have reduced the amount of code we had to write. However, for the sake of clarity, we have defined separate value sets for each field.
let's try this for the next field.

In [78]:
# 4. Care Pathway
# 4.1. First contact with specialised centre
vs_4_1 = ValueSet(
    elements=[Date],
    name="Date value set",
    description="Value set for date fields",
)

To implement the next field age at onset, we can use the `extend` method of the `ValueSet` class to create a new value set based on the dat value set but expanded by the values required by 5.1.:
- Antenatal
- At birth
- Undetermined

The `extend` function returns an expanded copy of the original value set with the new elements added. The original value set remains unchanged.

We can reuse this value set for the next field, age at diagnosis.

In [79]:
# 5. Disease history
# 5.1. Age at onset
vs_5_1 = vs_4_1.extend(
    new_name="Onset value set",
    value_set=ValueSet(["Antenatal", "At birth", "Undetermined"])
)
# 5.2. Age at diagnosis
vs_5_2 = vs_5_1

The fields in the following section diagnosis are all defined by using codings from code systems. We will define the value sets for these fields using the list of resources we defined above.

In [80]:
# 6. Diagnosis
# 6.1. Diagnosis of the rare disease
vs_6_1 = ValueSet(
    elements=[ORDO, ICD9, icd9cm, alpha, icd10],
    name="Value set for 6.1. Diagnosis of the rare disease",
    description="Value set for field 6.1. Diagnosis of the rare disease of the ERDRI CDS data model in section 6. Diagnosis",
)
# 6.2. Genetic diagnosis
vs_6_2 = ValueSet(
    elements=[HGVS, HGNC, OMIM],
    name="Value set for 6.2. Genetic diagnosis",
    description="Value set for field 6.2. Genetic diagnosis of the ERDRI CDS data model in section 6. Diagnosis",
)
# 6.3. Undiagnosed case
vs_6_3 = ValueSet(
    elements=[HPO, HGVS],
    name="Value set for 6.3. Undiagnosed case",
    description="Value set for field 6.3. Undiagnosed case of the ERDRI CDS data model in section 6. Diagnosis",
)

### 1.3 Define the `DataModel` object
After defining value sets for each field in the ERDRI CDS data model, we can now define the `DataModel` object. The `DataModel` object is the main object that represents the data model. 

The `DataField` constructor automatically handles turning the name of a field into a valid id in most cases. but sometimes this does not work. In any case, if you wish you can set a custom id for a field by passing it as the `id` parameter to the `DataField` constructor. Read the `DataField` carefully to adhere to the rulnaming rules.

In [81]:
from phenopacket_mapper.data_standards import DataModel, DataField

In [82]:
erdri_cds_data_model = DataModel(
    data_model_name="ERDRI CDS",
    resources=resources,
    fields=(
        # 1. Pseudonym
        # 1.1. Pseudonym
        DataField(section="1. Pseudonym", ordinal="1.1", name="Pseudonym", value_set=vs_1_1, required=True),

        # 2. Personal information
        # 2.1. Date of Birth
        DataField(section="2. Personal information", ordinal="2.1", name="Date of Birth", value_set=vs_2_1, required=True),
        # 2.2. Sex
        DataField(section="2. Personal information", ordinal="2.2", name="Sex", value_set=vs_2_2, required=True),
        
        # 3. Patient Status
        # 3.1. Patient's status
        DataField(section="3. Patient Status", ordinal="3.1", name="Patient's status", value_set=vs_3_1, required=True),
        # 3.2. Date of death
        DataField(section="3. Patient Status", ordinal="3.2", name="Date of death", value_set=vs_3_2, required=False),
        
        # 4. Care Pathway
        # 4.1. First contact with specialised centre
        DataField(section="4. Care Pathway", ordinal="4.1", name="First contact with specialised centre", value_set=vs_4_1),
        
        # 5. Disease history
        # 5.1. Age at onset
        DataField(section="5. Disease history", ordinal="5.1", name="Age at onset", value_set=vs_5_1),
        # 5.2. Age at diagnosis
        DataField(section="5. Disease history", ordinal="5.2", name="Age at diagnosis", value_set=vs_5_2),
        
        # 6. Diagnosis
        # 6.1. Diagnosis of the rare disease
        DataField(section="6. Diagnosis", ordinal="6.1", name="Diagnosis of the rare disease", value_set=vs_6_1),
        # 6.2. Genetic diagnosis
        DataField(section="6. Diagnosis", ordinal="6.2", name="Genetic diagnosis", value_set=vs_6_2),
        # 6.3. Undiagnosed case
        DataField(section="6. Diagnosis", ordinal="6.3", name="Undiagnosed case", value_set=vs_6_3),
    )
)

In [83]:
print(erdri_cds_data_model)

DataModel(name=ERDRI CDS
	DataField(
		id: pseudonym,
		section: 1. Pseudonym,
		ordinal, name: (1.1,  Pseudonym),
		value_set: ValueSet(elements=[<class 'str'>], name='Value set for 1.1. Pseudonym', description='Value set for field 1.1. Pseudonym of the ERDRI CDS data model in section 1. Pseudonym'), required: True,
		specification: 
	)
	DataField(
		id: date_of_birth,
		section: 2. Personal information,
		ordinal, name: (2.1,  Date of Birth),
		value_set: ValueSet(elements=[<class 'phenopacket_mapper.data_standards.date.Date'>], name='Value set for 2.1. Date of Birth', description='Value set for field 2.1. Date of Birth of the ERDRI CDS data model in section 2. Personal information'), required: True,
		specification: 
	)
	DataField(
		id: sex,
		section: 2. Personal information,
		ordinal, name: (2.2,  Sex),
		value_set: ValueSet(elements=['Female', 'Male', 'Undetermined', 'Foetus (Unknown)'], name='Value set for 2.2. Sex', description='Value set for field 2.2. Sex of the ERDRI CDS d

We can now access specific `DataField`s of the `DataModel` by using their id. 

E.g., we can get the date of birth field as follows:

In [84]:
erdri_cds_data_model.date_of_birth

DataField(name='Date of Birth', value_set=ValueSet(elements=[<class 'phenopacket_mapper.data_standards.date.Date'>], name='Value set for 2.1. Date of Birth', description='Value set for field 2.1. Date of Birth of the ERDRI CDS data model in section 2. Personal information'), id='date_of_birth', description='', section='2. Personal information', required=True, specification='', ordinal='2.1')

## 2. Load data using the ERDRI CDS Data Model

In [85]:
print(erdri_cds_data_model.get_field_ids())

['pseudonym', 'date_of_birth', 'sex', 'patient_s_status', 'date_of_death', 'first_contact_with_specialised_centre', 'age_at_onset', 'age_at_diagnosis', 'diagnosis_of_the_rare_disease', 'genetic_diagnosis', 'undiagnosed_case']


In [86]:
from phenopacket_mapper.pipeline import load_data_using_data_model
from pathlib import Path

In [87]:
data_path = Path('../res/test_data/erdri/erdri_cds_test_data.xlsx')

In [88]:
ds = erdri_cds_data_model.load_data(
    path = data_path,
    pseudonym_column="1.1. Pseudonym",
    date_of_birth_column= "2.1. Date of Birth",
    sex_column= "2.2. Sex",
    patient_s_status_column= "3.1. Patient's status",
    date_of_death_column= "3.2. Date of death",
    first_contact_with_specialised_centre_column= "4.1. First contact with specialised centre",
    age_at_onset_column= "5.1. Age at onset",
    age_at_diagnosis_column= "5.2. Age at diagnosis",
    diagnosis_of_the_rare_disease_column= "6.1. Diagnosis of the rare disease",
    genetic_diagnosis_column= "6.2. Genetic diagnosis",
    undiagnosed_case_column= None,
    
    compliance='soft'  # 'soft' or 'hard'
)

C:\Users\filip\OneDrive\Documents\dataspell\phenopacket_mapper\src\phenopacket_mapper\data_standards\data_model.py:358: UserWarning: Required fields are missing in the instance. (row 0) 
(missing_fields=undiagnosed_case)
  warnings.warn(error_msg)
C:\Users\filip\OneDrive\Documents\dataspell\phenopacket_mapper\src\phenopacket_mapper\data_standards\data_model.py:358: UserWarning: Required fields are missing in the instance. (row 1) 
(missing_fields=undiagnosed_case)
  warnings.warn(error_msg)
C:\Users\filip\OneDrive\Documents\dataspell\phenopacket_mapper\src\phenopacket_mapper\data_standards\data_model.py:358: UserWarning: Required fields are missing in the instance. (row 2) 
(missing_fields=undiagnosed_case)
  warnings.warn(error_msg)
C:\Users\filip\OneDrive\Documents\dataspell\phenopacket_mapper\src\phenopacket_mapper\data_standards\data_model.py:358: UserWarning: Required fields are missing in the instance. (row 3) 
(missing_fields=undiagnosed_case)
  warnings.warn(error_msg)
C:\Users

In [89]:
ds.head(20)

,pseudonym,date_of_birth,sex,patient_s_status,date_of_death,first_contact_with_specialised_centre,age_at_onset,age_at_diagnosis,diagnosis_of_the_rare_disease,genetic_diagnosis,undiagnosed_case
0,patient0,2002-02-00T00:00:00Z,Female,Alive,None,2019-00-00T00:00:00Z,Antenatal,Antenatal,ORPHA:206638 (),OMIM:614106 (),None
1,patient1,1979-06-17T00:00:00Z,Male,Dead,2010-02-00T00:00:00Z,2019-00-00T00:00:00Z,At birth,At birth,ICD9:781 (),OMIM:614106 (),None
2,patient2,2000-00-00T00:00:00Z,Male,Alive,None,2020-00-00T00:00:00Z,2005-12-07T00:00:00Z,2005-12-07T00:00:00Z,ORPHA:206638 (),OMIM:614106 (),None
3,patient3,2003-01-07T00:00:00Z,Female,Alive,None,2023-00-00T00:00:00Z,2009-07-22T00:00:00Z,2010-07-22T00:00:00Z,ORPHA:206638 (),OMIM:614106 (),None
4,patient4,2004-08-02T00:00:00Z,Undetermined,Lost in follow-up,None,2020-00-00T00:00:00Z,Undetermined,Undetermined,ORPHA:206638 (),None,None
5,patient5,1923-04-08T00:00:00Z,Foetus (Unknown),Opted-out,None,2021-00-00T00:00:00Z,Undetermined,Undetermined,ORPHA:206638 (),OMIM:614106 (),None
6,patient6,1999-00-00T00:00:00Z,False,Alive,None,2019-00-00T00:00:00Z,2017-00-00T00:00:00Z,2018-00-00T00:00:00Z,ORPHA:206638 (),None,None
7,patient7,2003-12-00T00:00:00Z,m,Alive,None,2024-00-00T00:00:00Z,2020-05-00T00:00:00Z,2020-05-00T00:00:00Z,ORPHA:206638 (),OMIM:614106 (),None
8,patient8,1979-09-00T00:00:00Z,Male,Dead,2017-05-31T00:00:00Z,2022-00-00T00:00:00Z,2009-00-00T00:00:00Z,2009-00-00T00:00:00Z,ORPHA:206638 (),OMIM:614106 (),None
9,patient9,2002-00-00T00:00:00Z,Female,Alive,None,2019-00-00T00:00:00Z,2024-00-00T00:00:00Z,2024-00-00T00:00:00Z,None,OMIM:614106 (),None


## 3. Preprocessing the data

It is important to preprocess the data to adhere to the Phenopacket schema. 

E.g.: The `phenopackets.VitalStatus` expects an enum value out of  'ALIVE', 'DECEASED', and 'UNKNOWN_STATUS' for the `phenopackets.VitalStatus.status` field. But the ERDRI CDS defined it's data set as 'Alive', 'Dead', 'Lost in follow-up', 'Opted-out'. So we need to map these values to the expected ones.

In [90]:
ds.preprocess(
    fields=erdri_cds_data_model.patient_s_status,
    mapping={
        "Alive": "ALIVE",
        "Dead": "DECEASED",
        "Lost in follow-up": "UNKNOWN_STATUS",
        "Opted-out": "UNKNOWN_STATUS"
    })

ds.preprocess(
    fields=erdri_cds_data_model.sex,
    mapping={
        'Female': 'FEMALE',
        'Male': 'MALE',
        'Undetermined': 'UNKNOWN_SEX',
        'Foetus (Unknown)': 'UNKNOWN_SEX',
        'm': 'MALE',
        False: 'FEMALE',
    }
)

def preprocess_age_at_diagnosis(values):
    age_at_diagnosis = values['age_at_diagnosis']
    date_of_birth = values['date_of_birth']
    if age_at_diagnosis == 'At birth' or age_at_diagnosis == 'Antenatal':
        return date_of_birth
    elif age_at_diagnosis == 'Undetermined':
        return None
    else:
        return age_at_diagnosis

ds.head(20)

,pseudonym,date_of_birth,sex,patient_s_status,date_of_death,first_contact_with_specialised_centre,age_at_onset,age_at_diagnosis,diagnosis_of_the_rare_disease,genetic_diagnosis,undiagnosed_case
0,patient0,2002-02-00T00:00:00Z,FEMALE,ALIVE,None,2019-00-00T00:00:00Z,Antenatal,Antenatal,ORPHA:206638 (),OMIM:614106 (),None
1,patient1,1979-06-17T00:00:00Z,MALE,DECEASED,2010-02-00T00:00:00Z,2019-00-00T00:00:00Z,At birth,At birth,ICD9:781 (),OMIM:614106 (),None
2,patient2,2000-00-00T00:00:00Z,MALE,ALIVE,None,2020-00-00T00:00:00Z,2005-12-07T00:00:00Z,2005-12-07T00:00:00Z,ORPHA:206638 (),OMIM:614106 (),None
3,patient3,2003-01-07T00:00:00Z,FEMALE,ALIVE,None,2023-00-00T00:00:00Z,2009-07-22T00:00:00Z,2010-07-22T00:00:00Z,ORPHA:206638 (),OMIM:614106 (),None
4,patient4,2004-08-02T00:00:00Z,UNKNOWN_SEX,UNKNOWN_STATUS,None,2020-00-00T00:00:00Z,Undetermined,Undetermined,ORPHA:206638 (),None,None
5,patient5,1923-04-08T00:00:00Z,UNKNOWN_SEX,UNKNOWN_STATUS,None,2021-00-00T00:00:00Z,Undetermined,Undetermined,ORPHA:206638 (),OMIM:614106 (),None
6,patient6,1999-00-00T00:00:00Z,FEMALE,ALIVE,None,2019-00-00T00:00:00Z,2017-00-00T00:00:00Z,2018-00-00T00:00:00Z,ORPHA:206638 (),None,None
7,patient7,2003-12-00T00:00:00Z,MALE,ALIVE,None,2024-00-00T00:00:00Z,2020-05-00T00:00:00Z,2020-05-00T00:00:00Z,ORPHA:206638 (),OMIM:614106 (),None
8,patient8,1979-09-00T00:00:00Z,MALE,DECEASED,2017-05-31T00:00:00Z,2022-00-00T00:00:00Z,2009-00-00T00:00:00Z,2009-00-00T00:00:00Z,ORPHA:206638 (),OMIM:614106 (),None
9,patient9,2002-00-00T00:00:00Z,FEMALE,ALIVE,None,2019-00-00T00:00:00Z,2024-00-00T00:00:00Z,2024-00-00T00:00:00Z,None,OMIM:614106 (),None


## 4. Defining the mapping to Phenopackets

In [91]:
import phenopackets

from phenopacket_mapper import PhenopacketMapper
from phenopacket_mapper.mapping import PhenopacketElement

In [92]:
added_fields = ['pseudonym', 'date_of_birth', 'patient_s_status', 'date_of_death', 'sex', 'diagnosis_of_the_rare_disease', 'age_at_diagnosis']

mapper = PhenopacketMapper(
    data_model=erdri_cds_data_model,
    id=erdri_cds_data_model.pseudonym,
    subject=PhenopacketElement(
        phenopacket_element=phenopackets.Individual,
        id=erdri_cds_data_model.pseudonym,
        date_of_birth=erdri_cds_data_model.date_of_birth,
        vital_status=PhenopacketElement(
            phenopacket_element=phenopackets.VitalStatus,
            status=erdri_cds_data_model.patient_s_status,
            time_of_death=PhenopacketElement(
                phenopacket_element=phenopackets.TimeElement,
                timestamp=erdri_cds_data_model.date_of_death
            )
        ),
        sex=erdri_cds_data_model.sex,
    ),
    diseases=[
        PhenopacketElement(
            phenopacket_element=phenopackets.Disease,
            term=erdri_cds_data_model.diagnosis_of_the_rare_disease,
            # TODO: preprocess to remove the strings
            # onset=PhenopacketElement(
            #     phenopacket_element=phenopackets.TimeElement,
            #     timestamp=erdri_cds_data_model.age_at_diagnosis
            # )
        )
    ],
    interpretations=[
        PhenopacketElement(
            phenopacket_element=phenopackets.Interpretation,
            diagnosis=PhenopacketElement(
                phenopacket_element=phenopackets.Diagnosis,
                disease=erdri_cds_data_model.diagnosis_of_the_rare_disease,
            )
        )  
    ],
    
)

## 5. Perform the mapping

In [93]:
phenopackets_list = mapper.map(ds)

In [94]:
print(phenopackets_list[1])

id: "patient1"
subject {
  id: "patient1"
  date_of_birth {
    seconds: 298425600
  }
  vital_status {
    status: DECEASED
    time_of_death {
      timestamp {
        seconds: 1264982400
      }
    }
  }
  sex: MALE
}
interpretations {
  diagnosis {
    disease {
      id: "ICD9:781 ()"
    }
  }
}
diseases {
  term {
    id: "ICD9:781 ()"
  }
}



## 6. Write the Phenopackets to file

In [95]:
from phenopacket_mapper.pipeline import write

In [96]:
output_path = Path('../res/test_data/erdri/output/')

write(phenopackets_list, out_dir=output_path)

## 7. Outlook

We want to implement a method like this:

In [97]:
# resources = [SNOMED_CT, ORDO, ]  # etc.
# ds = load_dataset(
#     path="...",
#     infer_data_model=True,
# )
# PhenopacketMapper.map(
#     data=ds,
#     resources=resources,
#     id=...
# )

TODO: debug loading of data properly

#### Continuance of the discussion raised by field 2.2. above

For the sake of completeness, we could use SNOMED Clinical Terms (SNOMED CT) to encode this variable using a value set containg:
- [SNOMED:248152002](https://browser.ihtsdotools.org/?perspective=full&conceptId1=248152002&edition=MAIN/2024-09-01&release=&languages=en) Female
- [SNOMED:248153007](https://browser.ihtsdotools.org/?perspective=full&conceptId1=248153007&edition=MAIN/2024-09-01&release=&languages=en) Male
- [SNOMED:373068000, SNOMED: 734000001](https://browser.ihtsdotools.org/?perspective=full&conceptId1=404684003&edition=MAIN/2024-09-01&release=&languages=en) Undetermined biological sex
- [SNOMED:303112003, SNOMED:373068000, SNOMED: 734000001](https://browser.ihtsdotools.org/?perspective=full&conceptId1=303112003&edition=MAIN/2024-09-01&release=&languages=en) Fetal period, biological sex unknown

To include multiple codings as a single value, one can use a `CodeableConcept` (`phenopacket_mapper.data_standards.code.CodeableConcept`) object. This object can contain multiple codings, each with a different code system if so wanted.

E.g.:

In [98]:
from phenopacket_mapper.data_standards import Coding, CodeableConcept
from phenopacket_mapper.data_standards.code_system import SNOMED_CT

sct_coding_female = Coding(code="248152002", system=SNOMED_CT, display="Female (finding)")
sct_coding_male = Coding(code="248153007", system=SNOMED_CT, display="Male (finding)")
sct_coding_undetermined = Coding(code="373068000", system=SNOMED_CT, display="Undetermined (qualifier value)")
sct_coding_bio_sex = Coding(code="734000001", system=SNOMED_CT, display="Biological sex (property) (qualifier value)")
sct_coding_foetus = Coding(code="303112003", system=SNOMED_CT, display="Fetal period (qualifier value)")

cc_und_bio_sex = CodeableConcept(coding=[sct_coding_undetermined, sct_coding_bio_sex], text="Undetermined biological sex")
cc_foetus = CodeableConcept(coding=[sct_coding_foetus, sct_coding_undetermined, sct_coding_bio_sex], text="Fetal period, biological sex unknown")

vs_bio_sex = ValueSet(elements=[sct_coding_female, sct_coding_male, cc_und_bio_sex, cc_foetus], name="", description="")

In [99]:
phenopackets.TimeElement()

In [100]:
date = Date(year=2024, month=9, day=12)
phenopackets.Phenopacket(
    id="1",
    subject=phenopackets.Individual(
        id="1",
        date_of_birth=date.protobuf_timestamp(),
    ),
    diseases=[
        phenopackets.Disease(
            onset=phenopackets.TimeElement(
                timestamp=date.protobuf_timestamp()
            )
        )
    ],
)

id: "1"
subject {
  id: "1"
  date_of_birth {
    seconds: 1726099200
  }
}
diseases {
  onset {
    timestamp {
      seconds: 1726099200
    }
  }
}